<a href="https://colab.research.google.com/github/4renginy/Module16-Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-02-01 18:08:48--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-02-01 18:08:49 (6.59 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(30)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

### Create DataFrames to match tables

In [6]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
vine_df = df.select(["review_id","star_rating", "helpful_votes", "total_votes","vine","verified_purchase" ])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|                Y|
|R2HBOEM8LE9928|          5|            0|          0|   N|                Y|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|                Y|
|R1EBPM82ENI67M|          1|            0|          0|   N|                Y|
|R372S58V6D11AT|          5|            1|          1|   N|                Y|
|R1A4514XOYI1PD|          5|            1|          1|   N|                Y|
|R20D9EHB7N20V6|          5|            0|          0|   N|                Y|
|R1WUTD8MVSROJU|          5|            0|          0|   N|                Y|
|R1QCYLT25812DM|          4|            0|          0|   N|                Y|
| R904DQPBCEM7A|          4|            0|          0|   N|     

In [7]:
# Filter total votes by votes over or equal to 20
total_votes_df= vine_df.filter("total_votes>=20")
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R227GSNWI6BSZV|          1|           20|         20|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|     

In [8]:
total_helpful_votes_df= total_votes_df.filter("helpful_votes/total_votes>=0.5")
total_helpful_votes_df.count()


50753

In [9]:
total_helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R227GSNWI6BSZV|          1|           20|         20|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|     

In [21]:
#Total count of five star ratings
total_fivestar_df=total_helpful_votes_df.filter(total_helpful_votes_df["star_rating"]==5)
total_fivestar_df.count()

23497

In [22]:
#Count of non vine club ratings
nonvinemem_df = total_helpful_votes_df.filter(total_helpful_votes_df["vine"]=="N")
nonvinemem_df.count()

49673

In [23]:
# Five star rating count for non vine club members
fivestar_nonvinemem=nonvinemem_df[nonvinemem_df["star_rating"]==5].count()
fivestar_nonvinemem

23043

In [24]:
#Count of vine club member ratings
vinemem_df= total_helpful_votes_df.filter(total_helpful_votes_df["vine"]=="Y")
vinemem_df.count()

1080

In [25]:
# Count of five star ratings of vine club members
fivestar_vinemem=vinemem_df[vinemem_df["star_rating"]==5].count()
fivestar_vinemem

454

In [26]:
#Five star percentage for vine club members
fivestar_vinemem_percent= float(vinemem_df[vinemem_df["star_rating"] ==5].count())/float(vinemem_df.count())*100
fivestar_vinemem_percent

42.03703703703704

In [29]:
#Five star percentage for non vine club members
fivestar_nonvinemem_percent= float(nonvinemem_df[nonvinemem_df["star_rating"] ==5].count())/float(nonvinemem_df.count())*100
fivestar_nonvinemem_percent

46.3893865882874

In [39]:
#Five star count for non vine members with verified purchase
fivestar_verifiedpurchase=nonvinemem_df[(nonvinemem_df["star_rating"]==5) & (nonvinemem_df["verified_purchase"]=="Y")].count()
fivestar_verifiedpurchase

12449

In [42]:
#Percentage of five star ratings for nonvine members with verified purchases
fivestar_nonvinemem_verified_percent= float(nonvinemem_df[(nonvinemem_df["star_rating"]==5) & (nonvinemem_df["verified_purchase"]=="Y")].count()/nonvinemem_df[nonvinemem_df["verified_purchase"]=="Y"].count())*100
fivestar_nonvinemem_verified_percent

48.06563706563707

In [52]:
#Percentage of five star for ratings for nonvine members with uverified purchases
fivestar_nonvinemem_unverified_percent= float(nonvinemem_df[(nonvinemem_df["star_rating"]==5) & (nonvinemem_df["verified_purchase"]=="N")].count()/nonvinemem_df[nonvinemem_df["verified_purchase"]=="N"].count())*100
fivestar_nonvinemem_unverified_percent

44.56315988726707

In [54]:
#Percentage of five tar ratings for vine members with verified purchase
fivestar_vinemem_verified_percent=float(vinemem_df[(vinemem_df["star_rating"]==5) & (vinemem_df["verified_purchase"]=="Y")].count()/ vinemem_df[vinemem_df["verified_purchase"]=="Y"].count())*100
fivestar_vinemem_verified_percent

100.0

In [61]:
#Percentage of five star ratings for vine members with unverified purchases
fivestar_vinemem_unverified_percent= float(vinemem_df[(vinemem_df["star_rating"]==5) & (vinemem_df["verified_purchase"]=="N")].count()/ vinemem_df[vinemem_df["verified_purchase"]=="N"].count())*100
fivestar_vinemem_unverified_percent

41.875580315691735